In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.optim as optim



#import dataset

training_data = datasets.FashionMNIST(root="data",
    train=True,
    download=True,
    transform=ToTensor())

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 11.9MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 176kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.28MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 12.8MB/s]


In [2]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")


Using cuda device


In [4]:
class FashionClassifier(nn.Module):
  def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)

        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.flat = nn.Flatten()

        self.fc3 = nn.Linear(6272, 512)
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(512, 10)

  def forward(self, x):

        x = self.act1(self.conv1(x))
        x = self.drop1(x)

        x = self.act2(self.conv2(x))

        x = self.pool2(x)

        x = self.flat(x)

        x = self.act3(self.fc3(x))
        x = self.drop3(x)

        x = self.fc4(x)
        return x



In [6]:
model = FashionClassifier().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

n_epochs = 20
for epoch in range(n_epochs):
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # forward, backward, and then weight update
        y_pred = model(inputs)
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    acc = 0
    count = 0
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        y_pred = model(inputs)
        acc += (torch.argmax(y_pred, 1) == labels).float().sum()
        count += len(labels)
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

Epoch 0: model accuracy 74.87%
Epoch 1: model accuracy 79.88%
Epoch 2: model accuracy 80.57%
Epoch 3: model accuracy 81.94%
Epoch 4: model accuracy 81.22%
Epoch 5: model accuracy 82.98%
Epoch 6: model accuracy 83.56%
Epoch 7: model accuracy 83.52%
Epoch 8: model accuracy 84.16%
Epoch 9: model accuracy 84.37%
Epoch 10: model accuracy 84.64%
Epoch 11: model accuracy 84.94%
Epoch 12: model accuracy 85.20%
Epoch 13: model accuracy 85.61%
Epoch 14: model accuracy 85.11%
Epoch 15: model accuracy 85.14%
Epoch 16: model accuracy 86.19%
Epoch 17: model accuracy 86.23%
Epoch 18: model accuracy 86.41%
Epoch 19: model accuracy 87.08%
